# 🔍 Notebook 07: Feature Selection

**Autor:** Gian  
**Fecha:** 2026-01-19  
**Objetivo:** Seleccionar las características más relevantes para alimentar el modelo de Machine Learning, reduciendo la dimensionalidad y el ruido.

---

## 📋 Contenido

1. Configuración del entorno
2. Carga de datos (Dataset Enriquecido)
3. Filtrado por Correlación (Multicolinealidad)
4. Selección por Mutual Information
5. Selección por Recursive Feature Elimination (RFE)
6. Importancia de Características (Random Forest)
7. Consolidación y Selección Final
8. Guardado del Dataset Reducido

---
## 1. Configuración del Entorno

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.feature_selection import mutual_info_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
sns.set_style("whitegrid")

np.random.seed(42)

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


---
## 2. Carga de Datos
Cargamos `06_data_engineered.csv` del paso anterior.

In [9]:
# Rutas
OUTPUT_PATH = Path("../../outputs/gian")
DATA_PATH = OUTPUT_PATH / "data" / "06_data_engineered.csv"
SELECTED_DATA_PATH = OUTPUT_PATH / "data"
REPORTS_PATH = OUTPUT_PATH / "reports"

# Cargar datos
df = pd.read_csv(DATA_PATH)

print(f"✅ Dataset cargado: {df.shape[0]:,} registros × {df.shape[1]} columnas")

✅ Dataset cargado: 9,701 registros × 104 columnas


---
## 3. Filtrado por Correlación (Multicolinealidad)
Eliminamos variables que estén altamente correlacionadas entre sí (rho > 0.9), manteniendo la que tenga mayor correlación con el target (si es posible) o simplemente la primera.

In [10]:
# Matriz de correlación
corr_matrix = df.corr(numeric_only=True).abs()

# Seleccionar triangulo superior
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Identificar columnas con correlación > 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

print(f"⚠️ Columnas a eliminar por alta correlación ({len(to_drop)}):")
print(to_drop[:10]) # Mostrar primeras 10

df_filtered = df.drop(columns=to_drop)
print(f"\n📊 Shape después de eliminar correlacionadas: {df_filtered.shape}")

⚠️ Columnas a eliminar por alta correlación (15):
['tasa_clics_marketing', 'riesgo_soporte_interaccion', 'valor_diario_cliente', 'log_errores_pago', 'log_tickets_soporte', 'log_antiguedad', 'borough_MANHATTAN', 'lineas_multiples_Sin servicio', 'seguridad_online_No internet service', 'respaldo_online_No internet service']

📊 Shape después de eliminar correlacionadas: (9701, 89)


---
## 4. Selección por Mutual Information
Calculamos la ganancia de información entre cada feature y el target (`cancelacion`).

In [11]:
X = df_filtered.select_dtypes(include=['number']).drop(columns=['cancelacion', 'cliente_id'], errors='ignore')
y = df_filtered['cancelacion']

# Calcular Mutual Information 
# (Usamos una muestra si el dataset fuera gigante, pero con 9k registros está bien usar todo)
mi_scores = mutual_info_classif(X, y, random_state=42)
mi_scores = pd.Series(mi_scores, name="MI_Score", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)

print("🏆 Top 10 Features por Mutual Information:")
print(mi_scores.head(10))

# Seleccionamos Top 30 (Arbitrario, para comparar)
top_mi_features = mi_scores.head(30).index.tolist()

🏆 Top 10 Features por Mutual Information:
score_riesgo              0.265987
dias_activos_semanales    0.265851
promedio_conexion         0.264081
conexiones_mensuales      0.263101
caracteristicas_usadas    0.261464
dias_ultima_conexion      0.245053
intensidad_uso            0.223572
tickets_soporte           0.192153
puntuacion_nps            0.174049
tasa_crecimiento_uso      0.148301
Name: MI_Score, dtype: float64


---
## 5. Selección por Recursive Feature Elimination (RFE)
Usamos Random Forest como estimador base para ir eliminando recursivamente las features menos importantes.

In [12]:
# Usamos un modelo 'ligero' para RFE
model_rfe = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42, n_jobs=-1)

# Seleccionar 20 mejores
rfe = RFE(model_rfe, n_features_to_select=20)
rfe.fit(X, y)

top_rfe_features = X.columns[rfe.support_].tolist()

print("🏆 Features seleccionadas por RFE:")
print(top_rfe_features)

🏆 Features seleccionadas por RFE:
['latitud', 'longitud', 'codigo_postal', 'antiguedad', 'cargo_mensual', 'ingresos_totales', 'errores_pago', 'score_riesgo', 'conexiones_mensuales', 'dias_activos_semanales', 'promedio_conexion', 'caracteristicas_usadas', 'tasa_crecimiento_uso', 'dias_ultima_conexion', 'tickets_soporte', 'tiempo_resolucion', 'puntuacion_csat', 'intensidad_uso', 'dias_desde_ultimo_contacto', 'ratio_carga_financiera']


---
## 6. Importancia de Características (Random Forest Completo)
Entrenamos un modelo robusto y extraemos `feature_importances_`.

In [13]:
rf_full = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_full.fit(X, y)

importances = pd.Series(rf_full.feature_importances_, index=X.columns).sort_values(ascending=False)

print("🏆 Top 10 Features por Random Forest Importance:")
print(importances.head(10))

top_rf_features = importances.head(25).index.tolist()

🏆 Top 10 Features por Random Forest Importance:
conexiones_mensuales      0.146455
promedio_conexion         0.117092
caracteristicas_usadas    0.116998
dias_activos_semanales    0.090268
score_riesgo              0.084496
dias_ultima_conexion      0.062194
tickets_soporte           0.043844
tasa_crecimiento_uso      0.034518
puntuacion_csat           0.027942
puntuacion_nps            0.025263
dtype: float64


---
## 7. Consolidación y Selección Final
Hacemos una votación: features que aparezcan en al menos 2 de los 3 métodos (MI, RFE, RF).

In [14]:
# Unimos todas las listas
all_selected = top_mi_features + top_rfe_features + top_rf_features

# Contamos frecuencia
from collections import Counter
counts = Counter(all_selected)

# Seleccionamos las que aparecen al menos en 2 métodos
final_features = [feat for feat, count in counts.items() if count >= 2]

# Asegurarnos de incluir el target y IDs si es necesario para el tracking
final_columns = ['cliente_id', 'cancelacion'] + final_features

print(f"✅ Número final de features seleccionadas (sin ID/Target): {len(final_features)}")
print("📋 Lista Final:", final_features)

✅ Número final de features seleccionadas (sin ID/Target): 24
📋 Lista Final: ['score_riesgo', 'dias_activos_semanales', 'promedio_conexion', 'conexiones_mensuales', 'caracteristicas_usadas', 'dias_ultima_conexion', 'intensidad_uso', 'tickets_soporte', 'puntuacion_nps', 'tasa_crecimiento_uso', 'puntuacion_csat', 'ratio_carga_financiera', 'tasa_apertura_email', 'errores_pago', 'antiguedad', 'ingresos_totales', 'latitud', 'cargo_mensual', 'tiempo_resolucion', 'longitud', 'codigo_postal', 'edad', 'dias_desde_ultimo_contacto', 'tiempo_sesion_promedio']


---
## 8. Guardado del Dataset Reducido

In [15]:
# Filtrar dataset
df_final = df[final_columns].copy()

# Guardar
output_file = SELECTED_DATA_PATH / "07_data_selected.csv"
df_final.to_csv(output_file, index=False)

# Guardar reporte
pd.DataFrame(final_features, columns=['Feature']).to_csv(REPORTS_PATH / "07_selected_features_list.csv", index=False)

print(f"💾 Dataset reducido guardado en: {output_file}")
print(f"📊 Nuevas dimensiones: {df_final.shape}")

💾 Dataset reducido guardado en: ../../outputs/gian/data/07_data_selected.csv
📊 Nuevas dimensiones: (9701, 26)
